In [1]:
## APIs

import pandas as pd

import csv
import requests


import config 
from config import API_URL, API_KEY, url, CSV_URL, news_url, BETA, IPO_URL

In [2]:
symbol = "AAPL" 

In [3]:
# url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=AMC&apikey=demo'
r = requests.get(url)
data = r.json()

time_series = data['Time Series (Daily)']
column_labels = ['1. open', '2. high', '3. low', '4. close', '5. adjusted close', '6. volume', '7. dividend amount', '8. split coefficient']

# create a DataFrame with the time series data and column labels
df = pd.DataFrame.from_dict(time_series, orient='index', dtype=float, columns=column_labels)

# sort the DataFrame by the date index in descending order (most recent date first)
df = df.sort_index(ascending=False)
df.columns = ['Open', 'High', 'Low', 'Close', 'Adjusted Close', 'Volume', 'Dividend Amount', 'Split Coefficient']

In [4]:
r = requests.get(news_url)
news_data = r.json()

In [5]:
df_news = pd.DataFrame(news_data['feed'])

In [6]:
# df2.loc[0,'topics']

In [7]:
df_news['ticker'] = df_news['ticker_sentiment'].apply(lambda x: [d['ticker'] for d in x])
df_news['ticker_sentiment_score'] = df_news['ticker_sentiment'].apply(lambda x: [d['ticker_sentiment_score'] for d in x])
df_news['ticker_sentiment_label'] = df_news['ticker_sentiment'].apply(lambda x: [d['ticker_sentiment_label'] for d in x])
df_news['relevance_score'] = df_news['ticker_sentiment'].apply(lambda x: [d['relevance_score'] for d in x])

df_news['topic'] = df_news['topics'].apply(lambda x: [d['topic'] for d in x])
df_news['relevance_score_topic'] = df_news['topics'].apply(lambda x: [d['relevance_score'] for d in x])


In [8]:
df_news = df_news[['relevance_score_topic','topic','relevance_score', 'ticker_sentiment_label','ticker_sentiment_score', 'overall_sentiment_label', 'overall_sentiment_score'  ]]
df_news.head(2)

,relevance_score_topic,topic,relevance_score,ticker_sentiment_label,ticker_sentiment_score,overall_sentiment_label,overall_sentiment_score
0,"[0.684621, 0.980922, 1.0, 0.158519]","[Financial Markets, Earnings, Technology, Econ...","[0.151539, 0.475838, 0.389636]","[Somewhat-Bullish, Bullish, Somewhat-Bullish]","[0.239377, 0.352506, 0.328757]",Somewhat-Bullish,0.236647
1,"[0.5, 0.983605, 0.5]","[Life Sciences, Blockchain, Manufacturing]","[0.114956, 0.114956, 0.114956, 0.114956, 0.282...","[Somewhat-Bullish, Somewhat-Bullish, Neutral, ...","[0.299005, 0.299005, 0.0, 0.299005, 0.132216, ...",Somewhat-Bullish,0.238351


In [9]:
# df.set_index('Date', inplace=True)
df = df.reset_index(level=0)

In [10]:

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        if row[0] == symbol:
            print(row)

['AAPL', 'Apple Inc', '2023-04-26', '2023-03-31', '1.43', 'USD']


In [11]:
'''import csv
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# CSV_URL = 'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&horizon=3month&apikey=demo'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)'''

'import csv\nimport requests\n\n# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key\n# CSV_URL = \'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&horizon=3month&apikey=demo\'\n\nwith requests.Session() as s:\n    download = s.get(CSV_URL)\n    decoded_content = download.content.decode(\'utf-8\')\n    cr = csv.reader(decoded_content.splitlines(), delimiter=\',\')\n    my_list = list(cr)\n    for row in my_list:\n        print(row)'

1. Market capitalization | Alpha Vantage

    value of a company's outstanding shares, can be used to detect changes in the company's overall value

In [12]:
# MarketCapitalization

params = {
    "function": "OVERVIEW",
    "symbol": symbol,
    "apikey": API_KEY,
}

response = requests.get(API_URL, params=params)
data = response.json()
market_cap = data["MarketCapitalization"]

params = {
    "function": "TIME_SERIES_DAILY_ADJUSTED",
    "symbol": symbol,
    "outputsize": "compact",
    "apikey": API_KEY,
}

response = requests.get(API_URL, params=params)
data = response.json()
prices = data["Time Series (Daily)"]

last_180_days = list(prices)[:365]

data_dict = {"date": [], "market_cap": []}

for date in last_180_days:
    market_cap_for_date = float(prices[date]["7. dividend amount"]) * float(prices[date]["5. adjusted close"])
    data_dict["date"].append(date)
    data_dict["market_cap"].append(market_cap_for_date)

df_MC = pd.DataFrame(data_dict)

'''import requests


params = {
    "function": "OVERVIEW",
    "symbol": symbol,
    "apikey": API_KEY,
}

response = requests.get(API_URL, params=params)
data = response.json()
market_cap = data["MarketCapitalization"]

params = {
    "function": "TIME_SERIES_DAILY_ADJUSTED",
    "symbol": symbol,
    "outputsize": "compact",
    "apikey": API_KEY,
}

response = requests.get(API_URL, params=params)
data = response.json()
prices = data["Time Series (Daily)"]

last_180_days = list(prices)[:180]

for date in last_180_days:
    market_cap_for_date = float(prices[date]["7. dividend amount"]) * float(prices[date]["5. adjusted close"])
    print(f"Market capitalization of {symbol} on {date}: {market_cap_for_date}")'''


'''
params = {
    "function": "OVERVIEW",
    "symbol": symbol,
    "apikey": API_KEY,
}

response = requests.get(API_URL, params=params)

data = response.json()

market_cap = data["MarketCapitalization"]
print(f"Market capitalization of {symbol}: {market_cap}")'''


'\nparams = {\n    "function": "OVERVIEW",\n    "symbol": symbol,\n    "apikey": API_KEY,\n}\n\nresponse = requests.get(API_URL, params=params)\n\ndata = response.json()\n\nmarket_cap = data["MarketCapitalization"]\nprint(f"Market capitalization of {symbol}: {market_cap}")'

2. Earnings per share (EPS) | 

    amount of a company's profit allocated to each outstanding share of stock, can be used to detect changes in a company's profitability

In [13]:
params = {
    "function": "EARNINGS",
    "symbol": symbol,
    "apikey": API_KEY,
}
import time
time.sleep(100)

response = requests.get(API_URL, params=params)
data = response.json()

eps_data = []

for quarter in data["quarterlyEarnings"]:
    if quarter["fiscalDateEnding"] >= "2022-02-19": # change the date to 180 days ago
        eps_data.append({
            "fiscalDateEnding": quarter["fiscalDateEnding"],
            "reportedEPS": quarter["reportedEPS"]
        })

df_eps = pd.DataFrame(eps_data)

'''
params = {
    "function": "OVERVIEW",
    "symbol": symbol,
    "apikey": API_KEY,
}

response = requests.get(API_URL, params=params)

data = response.json()

eps = data["EPS"]
print(f"Earnings per share of {symbol}: {eps}")'''

'''import requests


params = {
    "function": "EARNINGS",
    "symbol": symbol,
    "apikey": API_KEY,
}

response = requests.get(API_URL, params=params)
data = response.json()

for quarter in data["quarterlyEarnings"]:
    if quarter["fiscalDateEnding"] >= "2022-02-19": # change the date to 180 days ago
        eps = quarter["reportedEPS"]
        print(f"EPS for quarter ending {quarter['fiscalDateEnding']}: {eps}")'''


'import requests\n\n\nparams = {\n    "function": "EARNINGS",\n    "symbol": symbol,\n    "apikey": API_KEY,\n}\n\nresponse = requests.get(API_URL, params=params)\ndata = response.json()\n\nfor quarter in data["quarterlyEarnings"]:\n    if quarter["fiscalDateEnding"] >= "2022-02-19": # change the date to 180 days ago\n        eps = quarter["reportedEPS"]\n        print(f"EPS for quarter ending {quarter[\'fiscalDateEnding\']}: {eps}")'

3. Price-to-earnings ratio (P/E ratio)

    the price of a stock divided by its earnings per share, can be used to detect changes in a company's valuation

In [14]:
# time_series = data['Time Series (Daily)']

In [15]:
'''latest_date = list(data['Time Series (Daily)'].keys())[0]
pe_ratio = float(data['Time Series (Daily)'][latest_date]['8. Price']) / float(data['Time Series (Daily)'][latest_date]['4. Close'])
print(f"P/E Ratio on {latest_date}: {pe_ratio:.2f}")'''

'latest_date = list(data[\'Time Series (Daily)\'].keys())[0]\npe_ratio = float(data[\'Time Series (Daily)\'][latest_date][\'8. Price\']) / float(data[\'Time Series (Daily)\'][latest_date][\'4. Close\'])\nprint(f"P/E Ratio on {latest_date}: {pe_ratio:.2f}")'

In [16]:

params = {
    "function": "OVERVIEW",
    "symbol": symbol,
    "apikey": API_KEY,
}
import time
time.sleep(100)

response = requests.get(API_URL, params=params)
data = response.json()
pe_ratio = data["PERatio"]

params = {
    "function": "TIME_SERIES_DAILY_ADJUSTED",
    "symbol": symbol,
    "outputsize": "compact",
    "apikey": API_KEY,
}

response = requests.get(API_URL, params=params)
data = response.json()
prices = data["Time Series (Daily)"]

last_180_days = list(prices)[:180]

pe_ratios = []
for date in last_180_days:
    params = {
        "function": "OVERVIEW",
        "symbol": symbol,
        "apikey": API_KEY,
    }
    response = requests.get(API_URL, params=params)
    data = response.json()
    pe_ratio = data["PERatio"]
    pe_ratios.append({
        'date': date,
        'pe_ratio': pe_ratio,
    })

df_pe = pd.DataFrame(pe_ratios)



KeyError: 'PERatio'

In [ ]:
df_pe.head(2)

In [ ]:
'''
params = {
    "function": "OVERVIEW",
    "symbol": symbol,
    "apikey": API_KEY,
}

response = requests.get(API_URL, params=params)

data = response.json()

pe_ratio = data["PERatio"]
print(f"P/E ratio of {symbol}: {pe_ratio}")
'''


P/E ratio of AAPL: 25.9


4. Dividend yield

    percentage of a company's stock price paid out as dividends, can be used to detect changes in a company's financial health



In [ ]:
import yfinance as yf

symbol = 'AAPL'
stock = yf.Ticker(symbol)
dividend_yield = stock.info['dividendYield']
print(f"Dividend Yield: {dividend_yield}")

Exception: yfinance failed to decrypt Yahoo data response

In [ ]:
from datetime import datetime, timedelta


params = {
    "function": "OVERVIEW",
    "symbol": symbol,
    "apikey": API_KEY,
}

response = requests.get(API_URL, params=params)
data = response.json()
div_yield = data["DividendYield"]

params = {
    "function": "TIME_SERIES_DAILY_ADJUSTED",
    "symbol": symbol,
    "outputsize": "compact",
    "apikey": API_KEY,
}

response = requests.get(API_URL, params=params)
data = response.json()
prices = data["Time Series (Daily)"]

last_90_days = list(prices)[:90]

df_div = pd.DataFrame(columns=['date', 'div_yield'])

for date in last_90_days:
    div_yield_for_date = float(prices[date]["7. dividend amount"]) / float(prices[date]["5. adjusted close"])
    df = df.append({'date': date, 'div_yield': div_yield_for_date}, ignore_index=True)

'''
response = requests.get(API_URL, params=params)

data = response.json()

div_yield = data["DividendYield"]
print(f"Dividend yield of {symbol}: {div_yield}")'''


/var/folders/gr/77dww1vx4k945hzph39jwlc40000gn/T/ipykernel_16222/4157445135.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': date, 'div_yield': div_yield_for_date}, ignore_index=True)
/var/folders/gr/77dww1vx4k945hzph39jwlc40000gn/T/ipykernel_16222/4157445135.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': date, 'div_yield': div_yield_for_date}, ignore_index=True)
/var/folders/gr/77dww1vx4k945hzph39jwlc40000gn/T/ipykernel_16222/4157445135.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': date, 'div_yield': div_yield_for_date}, ignore_index=True)
/var/folders/gr/77dww1vx4k945hzph39jwlc40000gn/T/ipykernel_16222/4157445135.py:31: FutureWarning:

'\nresponse = requests.get(API_URL, params=params)\n\ndata = response.json()\n\ndiv_yield = data["DividendYield"]\nprint(f"Dividend yield of {symbol}: {div_yield}")'

5. BETA

In [ ]:
'''import requests

# Replace YOUR_API_KEY and SYMBOL with your actual API key and stock symbol from Alpha Vantage

response = requests.get(BETA)
data = response.json()

beta = data['BETA']
print(f"Beta: {beta}")'''


'import requests\n\n# Replace YOUR_API_KEY and SYMBOL with your actual API key and stock symbol from Alpha Vantage\n\nresponse = requests.get(BETA)\ndata = response.json()\n\nbeta = data[\'BETA\']\nprint(f"Beta: {beta}")'

6. Debt to Equity Ratio

In [ ]:
'''import requests

# Replace YOUR_API_KEY and SYMBOL with your actual API key and stock symbol from Intrinio
url = 'https://api-v2.intrinio.com/securities/{}/data_point/debt_to_equity?api_key=YOUR_API_KEY'.format(symbol)
response = requests.get(url)
data = response.json()

debt_to_equity_ratio = data['value']
print(f"Debt-to-Equity Ratio: {debt_to_equity_ratio}")'''


'import requests\n\n# Replace YOUR_API_KEY and SYMBOL with your actual API key and stock symbol from Intrinio\nurl = \'https://api-v2.intrinio.com/securities/{}/data_point/debt_to_equity?api_key=YOUR_API_KEY\'.format(symbol)\nresponse = requests.get(url)\ndata = response.json()\n\ndebt_to_equity_ratio = data[\'value\']\nprint(f"Debt-to-Equity Ratio: {debt_to_equity_ratio}")'

7. Price-to-Book (P/B) Ratio:

In [ ]:
'''import requests

# Replace YOUR_API_KEY and SYMBOL with your actual API key and stock symbol from Intrinio
url = 'https://api-v2.intrinio.com/securities/{}/data_point/book_value_per_share?api_key=YOUR_API_KEY'.format(SYMBOL)
response = requests.get(url)
data = response.json()

price = 100 # replace with actual stock price
pb_ratio = price / data['value']
print(f"P/B Ratio: {pb_ratio}")'''


'import requests\n\n# Replace YOUR_API_KEY and SYMBOL with your actual API key and stock symbol from Intrinio\nurl = \'https://api-v2.intrinio.com/securities/{}/data_point/book_value_per_share?api_key=YOUR_API_KEY\'.format(SYMBOL)\nresponse = requests.get(url)\ndata = response.json()\n\nprice = 100 # replace with actual stock price\npb_ratio = price / data[\'value\']\nprint(f"P/B Ratio: {pb_ratio}")'

IPO Calender

In [17]:
with requests.Session() as s:
    download = s.get(IPO_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)

['symbol', 'name', 'ipoDate', 'priceRangeLow', 'priceRangeHigh', 'currency', 'exchange']
['CTRMV', 'Castor Maritime Inc. Common Shares Ex-Distribution When Issued', '2023-02-21', '0', '0', 'USD', 'NASDAQ']
['TOROV', 'Toro Corp. Common Stock When-Issued', '2023-02-21', '0', '0', 'USD', 'NASDAQ']


In [18]:
# Merge features

df['tmp'] = 1
df_news['tmp'] = 1

df = pd.merge(df, df_news, on=['tmp'])
df = df.drop('tmp', axis=1)

In [19]:
df.head(2)

,index,Open,High,Low,Close,Adjusted Close,Volume,Dividend Amount,Split Coefficient,relevance_score_topic,topic,relevance_score,ticker_sentiment_label,ticker_sentiment_score,overall_sentiment_label,overall_sentiment_score
0,2023-02-17,5.32,5.53,5.21,5.24,5.24,27592365.0,0.0,1.0,"[0.684621, 0.980922, 1.0, 0.158519]","[Financial Markets, Earnings, Technology, Econ...","[0.151539, 0.475838, 0.389636]","[Somewhat-Bullish, Bullish, Somewhat-Bullish]","[0.239377, 0.352506, 0.328757]",Somewhat-Bullish,0.236647
1,2023-02-17,5.32,5.53,5.21,5.24,5.24,27592365.0,0.0,1.0,"[0.5, 0.983605, 0.5]","[Life Sciences, Blockchain, Manufacturing]","[0.114956, 0.114956, 0.114956, 0.114956, 0.282...","[Somewhat-Bullish, Somewhat-Bullish, Neutral, ...","[0.299005, 0.299005, 0.0, 0.299005, 0.132216, ...",Somewhat-Bullish,0.238351


In [20]:
df.overall_sentiment_label.value_counts()

Neutral             3000
Somewhat-Bullish    1600
Somewhat-Bearish     300
Bullish              100
Name: overall_sentiment_label, dtype: int64